In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
import datetime
from dateutil.rrule import rrule, DAILY

# deal with date

In [2]:
'''
三大法人交易情形統計資料自2012年5月1日起，提供交易日前三年之資料供查詢
，若需歷史資料，請填寫公開資料申請表申請  (只能查前三年)
三大法人交易情形統計資料自2011年12月19日起，揭露之交易資訊含鉅額交易量
'''
date_list = []
#時間抓取設定
start_date = datetime.date(2018, 1, 1)
end_date = datetime.date(2018, 1, 3)
for dt in rrule(DAILY, dtstart=start_date, until=end_date):
    date_list.append(dt.date())
#     print(dt.date())
#     print(str(dt.date()).split('-'))

print(date_list)

[datetime.date(2018, 1, 1), datetime.date(2018, 1, 2), datetime.date(2018, 1, 3)]


# get data

In [3]:
url = "https://www.taifex.com.tw/chinese/3/7_12_3.asp"

headers = {
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
}
post_data={
    'goday':'', 
    'DATA_DATE_Y': '2018',
    'DATA_DATE_M': '9',
    'DATA_DATE_D': '3',
    'syear': '2018',
    'smonth': '9',
    'sday': '3',
    'datestart': '2018/9/3',     #這個參數以上表示的是日期應該可以了解
#     'COMMODITY_ID': '',        #這個部分就是網頁契約的部分，空字串''表示全部
    'COMMODITY_ID': 'TXF'        #TXF表示臺股期貨，EXF表示電子期貨.....
          
}

time.sleep(2)

In [4]:
response = requests.post(url,headers=headers,data=post_data)    # 這個網站是透過post要求資料的
response.encoding = "utf-8"

soup = BeautifulSoup(response.text, "lxml")   # 網頁html

rows = soup.select('table.table_f tr')    # 搜尋期貨契約表格下的一列列tr資料
# table_f 是目標table的class name, 以tr為單位存進list
print(rows)

[<tr class="12bk">
<th align="middle" bgcolor="#6699CC" colspan="3" rowspan="2"><div align="center"></div></th>
<th align="middle" bgcolor="#6699CC" colspan="6"><div align="center">交易口數與契約金額</div></th>
<th align="middle" bgcolor="#6699CC" colspan="6"><div align="center">未平倉餘額</div></th>
</tr>, <tr class="12bk">
<th align="right" bgcolor="#6699CC" colspan="2"><div align="center">多方</div></th>
<th align="right" bgcolor="#6699CC" colspan="2"><div align="center">空方</div></th>
<th align="right" bgcolor="#6699CC" colspan="2"><div align="center">多空淨額</div></th>
<th align="right" bgcolor="#6699CC" colspan="2"><div align="center">多方</div></th>
<th align="right" bgcolor="#6699CC" colspan="2"><div align="center">空方</div></th>
<th align="right" bgcolor="#6699CC" colspan="2"><div align="center">多空淨額</div></th>
</tr>, <tr class="12bk">
<th bgcolor="#6699CC" width="15"><div align="center">序<br/>號</div></th>
<th bgcolor="#6699CC" width="30"><div align="center">商品<br/>名稱</div></th>
<th bgcolor="#6699CC

# test

In [5]:
indexs = ['日期','身分',
         '多方交易口數','多方契約金額',
         '空方交易口數','空方契約金額',
         '多空淨額口數','多空淨額契約金額',
         '未平倉多方交易口數','未平倉多方契約金額',
         '未平倉空方交易口數','未平倉空方契約金額',
         '未平倉多空淨額口數','未平倉多空淨額契約金額']
df = pd.DataFrame(columns=indexs)
df

,日期,身分,多方交易口數,多方契約金額,空方交易口數,空方契約金額,多空淨額口數,多空淨額契約金額,未平倉多方交易口數,未平倉多方契約金額,未平倉空方交易口數,未平倉空方契約金額,未平倉多空淨額口數,未平倉多空淨額契約金額


In [6]:
for d in date_list:
    date = str(d)
    for row in range(3,6):
        # deal with the data we scrapied and append it in to dataframe
        if row == 3:
            data_day = [rows[row].contents[i*2-1].get_text().replace(',',"").rstrip() for i in range(3,16)] 
        else:
            data_day = [rows[row].contents[i*2-1].get_text().replace(',',"").rstrip() for i in range(1,14)]
        data_day.insert(0,date)
        df = df.append(pd.Series(data_day, index = indexs), ignore_index=True)

print(df)

           日期   身分 多方交易口數     多方契約金額 空方交易口數     空方契約金額 多空淨額口數  多空淨額契約金額  \
0  2018-01-01  自營商  17845   39165563  19100   41907594  -1255  -2742031   
1  2018-01-01   投信      0          0    519    1134941   -519  -1134941   
2  2018-01-01   外資  62845  138017908  62403  137058274    442    959634   
3  2018-01-02  自營商  17845   39165563  19100   41907594  -1255  -2742031   
4  2018-01-02   投信      0          0    519    1134941   -519  -1134941   
5  2018-01-02   外資  62845  138017908  62403  137058274    442    959634   
6  2018-01-03  自營商  17845   39165563  19100   41907594  -1255  -2742031   
7  2018-01-03   投信      0          0    519    1134941   -519  -1134941   
8  2018-01-03   外資  62845  138017908  62403  137058274    442    959634   

  未平倉多方交易口數  未平倉多方契約金額 未平倉空方交易口數 未平倉空方契約金額 未平倉多空淨額口數 未平倉多空淨額契約金額  
0      9246   20160427     12388  27022579     -3142    -6862152  
1      2605    5696614     27575  60300710    -24970   -54604096  
2     73601  160917669     26679  58337102     4

In [7]:
# to see it's structure

# print(type(rows[3].get_text()))
# print(rows[3].get_text())
# print(rows[4].get_text())
# print(rows[4].contents[1].get_text())
# print(rows[5].get_text())

In [8]:
# date test

# A = datetime.datetime.now().date()
# print(A, type(A))

# B = datetime.datetime.now().strftime("%Y-%m-%d")
# print(B, type(B))

In [9]:
# another way

# table = soup.find_all('table')[2]
# table.encode('utf8')
# # print(table)

# df = pd.read_html(str(table),encoding='utf8')
# print(df)
# # print( tabulate(df[0], headers='keys', tablefmt='psql') )

In [5]:
a = '2018-08-20'
A = datetime.datetime.date(a)
print()

TypeError: descriptor 'date' requires a 'datetime.datetime' object but received a 'str'

# example

In [10]:
'''
每個商品名稱都有三個身分別，自營商、投信、外資
每一個身分別都是一個tr，所以一個商品有三個tr   用i來計算三個tr屬於同一個商品
第一個tr的第一個及第二個td都有rowspan="3"這個屬性
這表示每個商品第二及第三個tr都比第一個tr少兩個td
所以用i==1判斷是第一個tr，還是其它tr
另外contents會將空白or\n也當成是一筆資料所以contents[0]得到的是空字串
因此第一個td是contents[1]，第二個td是contents[3]，......依序下去
'''

i=0
for row in rows:    
    i += 1
    if i == 1:
        cell1 = row.contents[1].string   
        if cell1 and cell1.isdigit():
            print("{}-{}".format(cell1,row.contents[3].get_text()))
            print(row.contents[5].get_text(),row.contents[7].get_text())
    else:
        print(row.contents[1].get_text(),row.contents[3].get_text())

    if i == 3:
        print("--------------------------")
        i = 0

多方 空方
序號 商品名稱
--------------------------
1-臺股期貨
自營商 17,845 
投信 0 
外資 62,845 
--------------------------
投信 0 
外資 62,845 
--------------------------


In [11]:
# https://www.taifex.com.tw/chinese/3/7_12_3.asp

# https://data.gov.tw/dataset/11592

# https://quality.data.gov.tw/dq_download_xml.php?nid=11592&md5_url=08227086336ede0dc2b4d06e990e0758
# http://www.taifex.com.tw/data_gov/taifex_open_data.asp?data_name=MarketDataOfMajorInstitutionalTradersGeneralBytheDate